# Import Libraries and Clone Github

In [1]:
# Import Libraries
import os
import re
import time
import pandas as pd
import google.generativeai as genai

In [2]:
# Global Variables
CATEGORIES = [
        "Finance",
        "Production",
        "Reserves / Exploration / Acquisitions / Mergers / Divestments",
        "Environment / Regulatory / Geopolitics",
        "Alternative Energy / Lower Carbon",
        "Oil Price / Natural Gas Price / Gasoline Price"]
SENTIMENT_RESULTS_FILE_PATH = 'Prompt2_Sentiment_Analysis_Results.csv'

# Import Data

In [3]:
# Import all cleaned data files
invest_df1 = pd.read_csv('../02_Cleaned_Data/Investment_Research_Part1.csv')
invest_df2 = pd.read_csv('../02_Cleaned_Data/Investment_Research_Part2.csv')
proquest_df = pd.read_csv('../02_Cleaned_Data/ProQuest_Articles.csv')
earnings_presentations = pd.read_csv('../02_Cleaned_Data/Earnings_Presentations.csv')
earnings_qa = pd.read_csv('../02_Cleaned_Data/Earnings_QA.csv')
sec_df = pd.read_csv('../02_Cleaned_Data/SEC_Filings.csv')

# Merge into single df
text_df = pd.concat([invest_df1, invest_df2, proquest_df, sec_df, earnings_presentations, earnings_qa], ignore_index=True)
display(text_df.shape)
display(text_df.head())
display(text_df.tail())

(10053, 7)

,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
0,Investment Research,IR-1,MRO,2024-05-16,Marathon Oil Corporation,"Stock Report | May 16, 2024 | NYSESymbol: MRO ...",NaN
1,Investment Research,IR-2,EOG,2024-05-14,"EOG Resources, Inc.","Stock Report | May 14, 2024 | NYSESymbol: EOG ...",NaN
2,Investment Research,IR-3,EOG,2024-05-11,"EOG Resources, Inc.","Stock Report | May 11, 2024 | NYSESymbol: EOG ...",NaN
3,Investment Research,IR-4,DVN,2024-05-11,Devon Energy Corporation,"Stock Report | May 11, 2024 | NYSESymbol: DVN ...",NaN
4,Investment Research,IR-5,COP,2024-05-07,ConocoPhillips,"Stock Report | May 07, 2024 | NYSESymbol:\n\nC...",NaN


,Source,Unique_ID,Ticker,Date,Article Headline,Article Text,URL
10048,Earnings Call Q&A,EQ-338,XOM,2021-02-02,"Exxon Mobil Corporation, Q4 2020 Earnings Call...",Question and Answer\nOperator\n[Operator Instr...,NaN
10049,Earnings Call Q&A,EQ-339,COP,2021-02-02,"ConocoPhillips, Q4 2020 Earnings Call, Feb 02,...",Question and Answer\nOperator\n[Operator Instr...,NaN
10050,Earnings Call Q&A,EQ-340,EOG,2019-05-03,"EOG Resources, Inc., Q1 2019 Earnings Call, Ma...",Question and Answer\nOperator\n[Operator Instr...,NaN
10051,Earnings Call Q&A,EQ-341,SHEL,2019-05-02,"Royal Dutch Shell plc, Q1 2019 Earnings Call, ...",Question and Answer\nOperator\n[Operator Instr...,NaN
10052,Earnings Call Q&A,EQ-342,COP,2019-04-30,"ConocoPhillips, Q1 2019 Earnings Call, Apr 30,...",Question and Answer\nOperator\n[Operator Instr...,NaN


In [4]:
# Drop rows google gemini will not process
rows_to_drop = ['PQ-2840736837']
index_to_drops = text_df[text_df['Unique_ID'].isin(rows_to_drop)].index
text_df.drop(index_to_drops, inplace=True)
print(index_to_drops)

Index([5379], dtype='int64')


# Find or Create Results CSV

In [5]:
# Determine if the Sentiment Analysis Results file already exists
file_exists = os.path.isfile(SENTIMENT_RESULTS_FILE_PATH)

# Print the result
if file_exists:
    print(f"The file exists in the current directory.")
else:
    print(f"The file does not exist in the current directory.")

The file exists in the current directory.


In [6]:
# Create an empty file if the file does not exist
if not file_exists:
    # Copy text df and drop the text and headline column due to size
    empty_sentiment_df = text_df.copy()
    empty_sentiment_df = empty_sentiment_df.drop(['Article Text', 'Article Headline'], axis=1)

    for category in CATEGORIES:
        empty_sentiment_df[category] = ""
        empty_sentiment_df[category] = empty_sentiment_df[category].astype('object')

    # Display results
    display(empty_sentiment_df.head())

    # Save as CSV
    empty_sentiment_df.to_csv(SENTIMENT_RESULTS_FILE_PATH, index=False)

### NOTE: If you want to re-run the sentiment analysis, delete or archive the csv to create a blank one

# Sentiment Analysis
NOTE: Google gemini **currently** has a daily query limit of 1,500 requests per day.  As we have over 10,000 documents, the code will be designed to run over multiple days and pick up where we last left off.  After the code is run, the user will still need to manually download the csv and upload to github.

**Gemini Free Rate Limits**
*   15 RPM (requests per minute)
*   1 million TPM (tokens per minute)
*   1,500 RPD (requests per day)

In [7]:
# Set up Gemini. (API key needs to be in your environment variables)
key = 'GOOGLE_API_KEY'
GOOGLE_API_KEY = os.getenv(key)
genai.configure(api_key=GOOGLE_API_KEY)

# So it doesn't block the output
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_NONE",
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",},]

model = genai.GenerativeModel('gemini-1.5-flash-latest', safety_settings=safety_settings)

In [8]:
# Function to find the first empty row of the csv
def find_first_unique_id_with_empty_values(file_path, categories):
    """
    Finds the first unique ID where any of the specified columns have empty values in a CSV file.

    Args:
        file_path (str): The path to the CSV file.
        categories (list of str): List of column names to check for empty values.

    Returns:
        str: The first Unique_ID where any of the specified columns have empty values.
        None: If no such row is found.
    """

    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Iterate over each row to find the first unique ID with any empty values in the specified columns
    for index, row in df.iterrows():
        if row[categories].isnull().any() or (row[categories] == '').any():
            return row['Unique_ID']

    return None  # Return None if no such row is found

# Test function
unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)
print(unique_id)

PQ-2635684693


In [9]:
# Helper function to get Gemini query function inputs
def get_gemini_inputs(text_df, unique_id):
    """
    Retrieves information from the DataFrame based on the unique ID and outputs company, source, headline, and text.

    Args:
        text_df (pd.DataFrame): The DataFrame containing the text data.
        unique_id (str): The unique ID to search for.

    Returns:
        tuple: A tuple containing company, source, headline, and text.
    """
    # Find the row with the specified unique ID
    row = text_df[text_df['Unique_ID'] == unique_id]

    # Extract the required information
    company = row['Ticker'].values[0]
    source = row['Source'].values[0]
    headline = row['Article Headline'].values[0]
    text = row['Article Text'].values[0]

    return company, source, headline, text

# Test function
company, source, headline, text = get_gemini_inputs(text_df, unique_id)
print(f"Company: {company}\n")
print(f"Source: {source}\n")
print(f"Headline: {headline}\n")
print(f"Text:\n{text}")

Company: XOM

Source: ProQuest

Headline: Apple Exxon Mobil Rivian Stocks That Defined Week

Text:
Apple Inc.
Major Western companies are retreating from Russia . Apple said Tuesday it stopped selling iPhones and other products in Russia, saying it was "deeply concerned about the Russian invasion of Ukraine." Ukraine's vice prime minister on Feb. 25 asked Chief Executive Tim Cook to stop supplying products and services—including access to the App Store—to Russia. Other companies that made similar moves include Ford Motor Co. and Dell Technologies Inc. Apple shares gained 2.1% Wednesday.
Zoom Video Communications Inc.
The return of the office may be slowing Zoom's connection. The company's sales growth faltered in the fourth quarter, signaling that demand for its videoconferencing application is no longer as entrenched in daily life as it was earlier in the pandemic. Zoom's performance soared when Covid-19 treatments were largely unavailable and much of the world operated on virtual pla

In [10]:
# Function to query Gemini
def query_gemini(company, source, headline, text, model):
    """
    Query Gemini to perform sentiment analysis on text from various sources about a company.

    Parameters:
    company (str): The name of the company the text is about.
    source (str): The source of the text. Valid values are "Investment Research", "ProQuest", "SEC Filings", "Earnings Call Presentations", "Earnings Call Q&A".
    headline (str): The headline or title of the text.
    text (str): The body of the text to be analyzed.
    model: The model object used to generate content and analyze the text.

    Returns:
    str: The sentiment analysis results for predefined categories in the specified format.

    The function constructs a prompt based on the source of the text and performs sentiment analysis on the given text using the provided model.
    It analyzes the content across multiple predefined categories, determining the sentiment (Positive, Neutral, Negative) for each category.
    If a category is not mentioned or relevant based on the text content, it is marked as 'Neutral'.
    The final output is summarized in a specified format.
    """

    # Source Variables
    if source == "Investment Research":
        text_source = "an analyst report"
        text_source2 = "the analyst report"
    elif source == "ProQuest":
        text_source = "a news article"
        text_source2 = "the news article"
    elif source == "SEC Filings":
        text_source = "an SEC filing"
        text_source2 = "the SEC filing"
    elif source == "Earnings Call Presentations":
        text_source = "an earnings call presentation"
        text_source2 = "the earnings call presentation"
    elif source == "Earnings Call Q&A":
        text_source = "an earnings call Q&A session"
        text_source2 = "the earnings call Q&A session"

    # Prompt
    prompt = f"""
Given the text from {text_source} about {company}, analyze the content and perform sentiment analysis across multiple predefined categories.

Sentiment options:
  - Positive
  - Neutral
  - Negative

Categories:
  - Finance
  - Production
  - Reserves / Exploration / Acquisitions / Mergers / Divestments
  - Environment / Regulatory / Geopolitics
  - Alternative Energy / Lower Carbon
  - Oil Price / Natural Gas Price / Gasoline Price

Each category should be evaluated and given a sentiment output derived from the text.
If a category is not mentioned or relevant based on the text content, mark it as 'Neutral'.

Before giving your answer, explain your reasoning and reference the article.
After going through all the categories, provide a summary in the following format:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Example Output1:
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral

Example Output 2:
 - Finance - Negative
 - Production - Positive
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral
 - Environment / Regulatory / Geopolitics - Positive
 - Alternative Energy / Lower Carbon - Neutral
 - Oil Price / Natural Gas Price / Gasoline Price - Negative

Example Output 3:
 - Finance - Neutral
 - Production - Negative
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Positive
 - Environment / Regulatory / Geopolitics - Negative
 - Alternative Energy / Lower Carbon - Positive
 - Oil Price / Natural Gas Price / Gasoline Price - Neutral

Example Output 4:
 - Finance - Positive
 - Production - Neutral
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative
 - Environment / Regulatory / Geopolitics - Neutral
 - Alternative Energy / Lower Carbon - Negative
 - Oil Price / Natural Gas Price / Gasoline Price - Positive

Example Output 5:
 - Finance - Negative
 - Production - Positive
 - Reserves / Exploration / Acquisitions / Mergers / Divestments - Neutral
 - Environment / Regulatory / Geopolitics - Negative
 - Alternative Energy / Lower Carbon - Positive
 - Oil Price / Natural Gas Price / Gasoline Price - Negative

Make sure to use plain text, do not bold or bullet the output summary.

The text from {text_source2} is below:
{headline}
{text}

Remember to summarize your final answers in the following format exactly:
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment
- Category - Sentiment

Make sure to use plain text and stick to the given categories and sentiment options.
DO NOT bold or bullet the output summary.
    """

    # print(prompt)
    response = model.generate_content(prompt)
    return response.text

# Test function
response = query_gemini(company, source, headline, text, model)
print(response)

## Sentiment Analysis of XOM Article:

The article focuses on Exxon Mobil's response to the Russian invasion of Ukraine and its impact on the oil market. Here's a breakdown of the sentiment analysis:

**- Finance - Positive:**  The article mentions that Exxon Mobil shares rose 1.7% on Wednesday, likely due to the surge in oil prices. This indicates a positive sentiment towards the company's financial performance in the context of the rising oil market.

**- Production - Neutral:** While the article mentions Exxon Mobil's decision to halt operations at a multibillion-dollar oil-and-gas project in Russia, there's no direct information on the impact on their overall production. Therefore, the sentiment is neutral.

**- Reserves / Exploration / Acquisitions / Mergers / Divestments - Negative:** The article explicitly states that Exxon Mobil is halting operations and will make no further investments in Russia. This signifies a negative sentiment towards future exploration and development ac

In [11]:
# Function to parse text
def parse_sentiment(text, categories):
    """
    Parses a given text for specified categories and their sentiments.

    Args:
        text (str): The input text containing categories and their sentiments.
        categories (list of str): List of category names to look for in the text.

    Returns:
        dict or str: A dictionary with categories as keys and their corresponding sentiments as values,
                     or "Did not find all categories" if any sentiment is not Positive, Neutral, or Negative.
    """
    results = {}
    valid_sentiments = {"Positive", "Neutral", "Negative"}

    for category in categories:
        # Create a regex pattern to match the format "- Category - Sentiment"
        # The category name is escaped to handle any special characters
        pattern = rf"- {re.escape(category)} - (\w+)"

        # Search for the pattern in the text
        match = re.search(pattern, text)

        # If a match is found, extract the sentiment and add it to the results dictionary
        if match:
            sentiment = match.group(1)
            if sentiment not in valid_sentiments:
                return "Did not find all categories"
            results[category] = sentiment
        else:
            return "Did not find all categories"

    return results

# Test function
fail_text = """
- Finance - Positive
- Production - Neutral
- Reserves / Exploration / Acquisitions / Mergers / Divestments - Bad
- Environment / Regulatory / Geopolitics - Neutral
- Alternative Energy / Lower Carbon - Positive
- Oil Price / Natural Gas Price / Gasoline Price - Neutral
"""

sentiment_dict = parse_sentiment(response, CATEGORIES)
fail_sentiment = parse_sentiment("fail_text", CATEGORIES)
print(sentiment_dict)
print(fail_sentiment)

{'Finance': 'Positive', 'Production': 'Neutral', 'Reserves / Exploration / Acquisitions / Mergers / Divestments': 'Negative', 'Environment / Regulatory / Geopolitics': 'Neutral', 'Alternative Energy / Lower Carbon': 'Neutral', 'Oil Price / Natural Gas Price / Gasoline Price': 'Positive'}
Did not find all categories


In [12]:
# Function to update the csv
def update_csv(file_path, unique_id, sentiment_dict):
    """
    Updates the columns of a CSV file based on the unique ID and sentiment dictionary.

    Args:
        file_path (str): The path to the CSV file.
        unique_id (str): The unique ID of the row to be updated.
        sentiment_dict (dict): A dictionary with categories as keys and their corresponding sentiments as values.

    Returns:
        None
    """
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)

    # Find the index of the row with the specified unique ID
    row_index = df[df['Unique_ID'] == unique_id].index

    # Update the columns based on the sentiment dictionary
    for category, sentiment in sentiment_dict.items():
        df.loc[row_index, category] = sentiment

    # Save the updated DataFrame back to the CSV file
    df.to_csv(file_path, index=False)
    print(f"Row with Unique_ID '{unique_id}' has been updated.")

# Test function
update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict)

Row with Unique_ID 'PQ-2635684693' has been updated.


In [13]:
start_time = time.time()
unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)
count = 0
max_tries = 5

# Iterate through the CSV using the functions
while unique_id:
    retries = 0
    success = False

    # There are multiple errors that can happen, many of them simply need another try
    while retries < max_tries and not success:
        try:
            # Get gemini inputs
            company, source, headline, text = get_gemini_inputs(text_df, unique_id)

            # Query Gemini
            response = query_gemini(company, source, headline, text, model)

            # Parse text
            sentiment_dict = parse_sentiment(response, CATEGORIES)

            # Update the csv
            update_csv(SENTIMENT_RESULTS_FILE_PATH, unique_id, sentiment_dict)

            success = True

        except Exception as e:
            retries += 1
            print(f"Error encountered: {e}. Retry {retries}/{max_tries}")

            if retries >= max_tries:
                print(f"Failed to process unique_id {unique_id} after {max_tries} attempts. Stopping.")
                # Exit both loops
                unique_id = None
                break

    if not success:
        break

    # Get an update every 10 rows
    count += 1
    if count % 10 == 0:
        elapsed_time = time.time() - start_time
        minutes, seconds = divmod(elapsed_time, 60)
        print(f"Iteration: {count}, Elapsed Time: {int(minutes)} minutes and {seconds:.2f} seconds")

    # Find the next empty row
    unique_id = find_first_unique_id_with_empty_values(SENTIMENT_RESULTS_FILE_PATH, CATEGORIES)

    # Test print statements
    # print(unique_id)
    # print(f"Company: {company}\n")
    # print(f"Source: {source}\n")
    # print(f"Headline: {headline}\n")
    # print(f"Text:\n{text}")
    # print(response)
    # print(sentiment_dict)

Row with Unique_ID 'PQ-2635256694' has been updated.
Row with Unique_ID 'PQ-2635314949' has been updated.
Row with Unique_ID 'PQ-2635161746' has been updated.
Row with Unique_ID 'PQ-2634870869' has been updated.
Row with Unique_ID 'PQ-2512376328' has been updated.
Row with Unique_ID 'PQ-2464144160' has been updated.
Row with Unique_ID 'PQ-2460769247' has been updated.
Row with Unique_ID 'PQ-2460088266' has been updated.
Row with Unique_ID 'PQ-2458488855' has been updated.
Row with Unique_ID 'PQ-2458407133' has been updated.
Iteration: 10, Elapsed Time: 1 minutes and 59.63 seconds
Row with Unique_ID 'PQ-2467512336' has been updated.
Row with Unique_ID 'PQ-2467512046' has been updated.
Row with Unique_ID 'PQ-2458216645' has been updated.
Row with Unique_ID 'PQ-2457706001' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'PQ-2457420881' has been updated.
Row with Unique_ID 'PQ-2456396416' has been updated.
Row with Unique_ID 'PQ-2455

Row with Unique_ID 'PQ-2407487889' has been updated.
Row with Unique_ID 'PQ-2406797800' has been updated.
Row with Unique_ID 'PQ-2405760931' has been updated.
Row with Unique_ID 'PQ-2403387629' has been updated.
Row with Unique_ID 'PQ-2412431955' has been updated.
Iteration: 140, Elapsed Time: 27 minutes and 21.02 seconds
Row with Unique_ID 'PQ-2401965937' has been updated.
Row with Unique_ID 'PQ-2399826211' has been updated.
Row with Unique_ID 'PQ-2408381655' has been updated.
Row with Unique_ID 'PQ-2397179046' has been updated.
Row with Unique_ID 'PQ-2397129122' has been updated.
Row with Unique_ID 'PQ-2396988654' has been updated.
Row with Unique_ID 'PQ-2396875981' has been updated.
Row with Unique_ID 'PQ-2396295080' has been updated.
Row with Unique_ID 'PQ-2396303631' has been updated.
Row with Unique_ID 'PQ-2396284965' has been updated.
Iteration: 150, Elapsed Time: 29 minutes and 15.86 seconds
Row with Unique_ID 'PQ-2396198772' has been updated.
Row with Unique_ID 'PQ-2394691271'

Row with Unique_ID 'PQ-2635344606' has been updated.
Row with Unique_ID 'PQ-2635343189' has been updated.
Row with Unique_ID 'PQ-2624855836' has been updated.
Row with Unique_ID 'PQ-2624561453' has been updated.
Row with Unique_ID 'PQ-2624532259' has been updated.
Row with Unique_ID 'PQ-2624463729' has been updated.
Row with Unique_ID 'PQ-2624242198' has been updated.
Row with Unique_ID 'PQ-2624157749' has been updated.
Row with Unique_ID 'PQ-2624092487' has been updated.
Iteration: 280, Elapsed Time: 54 minutes and 16.03 seconds
Row with Unique_ID 'PQ-2624083488' has been updated.
Row with Unique_ID 'PQ-2624054309' has been updated.
Row with Unique_ID 'PQ-2623998915' has been updated.
Row with Unique_ID 'PQ-2623895215' has been updated.
Row with Unique_ID 'PQ-2623837782' has been updated.
Row with Unique_ID 'PQ-2623809189' has been updated.
Row with Unique_ID 'PQ-2623775996' has been updated.
Row with Unique_ID 'PQ-2623732613' has been updated.
Row with Unique_ID 'PQ-2623575854' has b

Row with Unique_ID 'PQ-2553162752' has been updated.
Row with Unique_ID 'PQ-2552977943' has been updated.
Row with Unique_ID 'PQ-2552837424' has been updated.
Row with Unique_ID 'PQ-2551427202' has been updated.
Row with Unique_ID 'PQ-2548654153' has been updated.
Row with Unique_ID 'PQ-2547889634' has been updated.
Iteration: 410, Elapsed Time: 79 minutes and 42.28 seconds
Row with Unique_ID 'PQ-2546868727' has been updated.
Row with Unique_ID 'PQ-2546868715' has been updated.
Row with Unique_ID 'PQ-2546770036' has been updated.
Row with Unique_ID 'PQ-2546775564' has been updated.
Row with Unique_ID 'PQ-2546689009' has been updated.
Row with Unique_ID 'PQ-2546598031' has been updated.
Row with Unique_ID 'PQ-2543901239' has been updated.
Row with Unique_ID 'PQ-2543700151' has been updated.
Row with Unique_ID 'PQ-2540764382' has been updated.
Row with Unique_ID 'PQ-2540046220' has been updated.
Iteration: 420, Elapsed Time: 81 minutes and 42.57 seconds
Row with Unique_ID 'PQ-2540027812'

Row with Unique_ID 'PQ-2493715668' has been updated.
Row with Unique_ID 'PQ-2493505019' has been updated.
Row with Unique_ID 'PQ-2505571935' has been updated.
Row with Unique_ID 'PQ-2491612337' has been updated.
Row with Unique_ID 'PQ-2490405032' has been updated.
Row with Unique_ID 'PQ-2489184288' has been updated.
Row with Unique_ID 'PQ-2488070034' has been updated.
Row with Unique_ID 'PQ-2487359190' has been updated.
Row with Unique_ID 'PQ-2487322583' has been updated.
Iteration: 550, Elapsed Time: 128 minutes and 35.04 seconds
Row with Unique_ID 'PQ-2487118910' has been updated.
Row with Unique_ID 'PQ-2486610930' has been updated.
Row with Unique_ID 'PQ-2486500044' has been updated.
Row with Unique_ID 'PQ-2486496971' has been updated.
Row with Unique_ID 'PQ-2485947715' has been updated.
Row with Unique_ID 'PQ-2485384511' has been updated.
Row with Unique_ID 'PQ-2485321890' has been updated.
Row with Unique_ID 'PQ-2485321172' has been updated.
Row with Unique_ID 'PQ-2484918239' has 

...

Row with Unique_ID 'PQ-2388169078' has been updated.
Row with Unique_ID 'PQ-2388168224' has been updated.
Row with Unique_ID 'PQ-2388168202' has been updated.
Iteration: 820, Elapsed Time: 176 minutes and 53.93 seconds
Row with Unique_ID 'PQ-2388168197' has been updated.
Row with Unique_ID 'PQ-2388168119' has been updated.
Row with Unique_ID 'PQ-2388167980' has been updated.
Row with Unique_ID 'PQ-2376024247' has been updated.
Row with Unique_ID 'PQ-2375662821' has been updated.
Row with Unique_ID 'PQ-2375580510' has been updated.
Row with Unique_ID 'PQ-2375523303' has been updated.
Row with Unique_ID 'PQ-2375517056' has been updated.
Row with Unique_ID 'PQ-2385089469' has been updated.
Row with Unique_ID 'PQ-2385089263' has been updated.
Iteration: 830, Elapsed Time: 178 minutes and 49.59 seconds
Row with Unique_ID 'PQ-2385087258' has been updated.
Row with Unique_ID 'PQ-2385087219' has been updated.
Row with Unique_ID 'PQ-2366423392' has been updated.
Row with Unique_ID 'PQ-235494175

Row with Unique_ID 'PQ-2486118769' has been updated.
Row with Unique_ID 'PQ-2483003353' has been updated.
Row with Unique_ID 'PQ-2480243210' has been updated.
Row with Unique_ID 'PQ-2476170254' has been updated.
Row with Unique_ID 'PQ-2475281929' has been updated.
Row with Unique_ID 'PQ-2472388169' has been updated.
Row with Unique_ID 'PQ-2471455919' has been updated.
Row with Unique_ID 'PQ-2464858006' has been updated.
Row with Unique_ID 'PQ-2459192716' has been updated.
Row with Unique_ID 'PQ-2456880308' has been updated.
Iteration: 960, Elapsed Time: 204 minutes and 12.93 seconds
Row with Unique_ID 'PQ-2455687820' has been updated.
Row with Unique_ID 'PQ-2455547594' has been updated.
Row with Unique_ID 'PQ-2455149667' has been updated.
Row with Unique_ID 'PQ-2448400168' has been updated.
Row with Unique_ID 'PQ-2456192722' has been updated.
Row with Unique_ID 'PQ-2456191912' has been updated.
Row with Unique_ID 'PQ-2456191624' has been updated.
Row with Unique_ID 'PQ-2456191510' has 

Row with Unique_ID 'PQ-2921870202' has been updated.
Iteration: 1090, Elapsed Time: 226 minutes and 53.15 seconds
Row with Unique_ID 'PQ-2919769187' has been updated.
Row with Unique_ID 'PQ-2919732568' has been updated.
Row with Unique_ID 'PQ-2919805855' has been updated.
Row with Unique_ID 'PQ-2916902845' has been updated.
Row with Unique_ID 'PQ-2928067560' has been updated.
Row with Unique_ID 'PQ-2927882057' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'PQ-2927732143' has been updated.
Row with Unique_ID 'PQ-2915859725' has been updated.
Row with Unique_ID 'PQ-2915809384' has been updated.
Row with Unique_ID 'PQ-2915479291' has been updated.
Iteration: 1100, Elapsed Time: 228 minutes and 42.31 seconds
Row with Unique_ID 'PQ-2914936625' has been updated.
Row with Unique_ID 'PQ-2914933985' has been updated.
Row with Unique_ID 'PQ-2912926016' has been updated.
Row with Unique_ID 'PQ-2912953218' has been updated.
Row with Unique

Row with Unique_ID 'PQ-2730875330' has been updated.
Row with Unique_ID 'PQ-2730864235' has been updated.
Row with Unique_ID 'PQ-2730530812' has been updated.
Row with Unique_ID 'PQ-2728549024' has been updated.
Row with Unique_ID 'PQ-2728485787' has been updated.
Row with Unique_ID 'PQ-2727610098' has been updated.
Iteration: 1230, Elapsed Time: 251 minutes and 47.68 seconds
Row with Unique_ID 'PQ-2726789233' has been updated.
Row with Unique_ID 'PQ-2725594900' has been updated.
Row with Unique_ID 'PQ-2725367743' has been updated.
Row with Unique_ID 'PQ-2725177870' has been updated.
Row with Unique_ID 'PQ-2720490019' has been updated.
Row with Unique_ID 'PQ-2720100676' has been updated.
Row with Unique_ID 'PQ-2718529781' has been updated.
Row with Unique_ID 'PQ-2718099647' has been updated.
Row with Unique_ID 'PQ-2716267371' has been updated.
Row with Unique_ID 'PQ-2716784619' has been updated.
Iteration: 1240, Elapsed Time: 253 minutes and 36.63 seconds
Row with Unique_ID 'PQ-2712368

Row with Unique_ID 'PQ-2515568056' has been updated.
Row with Unique_ID 'PQ-2514263173' has been updated.
Row with Unique_ID 'PQ-2513985953' has been updated.
Row with Unique_ID 'PQ-2513979406' has been updated.
Row with Unique_ID 'PQ-2509156574' has been updated.
Row with Unique_ID 'PQ-2508847070' has been updated.
Row with Unique_ID 'PQ-2508825441' has been updated.
Row with Unique_ID 'PQ-2497895328' has been updated.
Row with Unique_ID 'PQ-2491854937' has been updated.
Row with Unique_ID 'PQ-2490148625' has been updated.
Iteration: 1370, Elapsed Time: 280 minutes and 57.04 seconds
Row with Unique_ID 'PQ-2489046366' has been updated.
Error encountered: 'str' object has no attribute 'items'. Retry 1/5
Row with Unique_ID 'PQ-2485384535' has been updated.
Row with Unique_ID 'PQ-2480011920' has been updated.
Row with Unique_ID 'PQ-2480011792' has been updated.
Row with Unique_ID 'PQ-2479906884' has been updated.
Row with Unique_ID 'PQ-2479906122' has been updated.
Row with Unique_ID 'PQ-